In [1]:
library(glmnet)
library(matrixStats)
library(abind)
library(pracma)
library(matrixcalc)
library(TCA)

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-18


Attaching package: ‘pracma’

The following objects are masked from ‘package:Matrix’:

    expm, lu, tril, triu



In [186]:
"
=================================================
This file is used to generate data from specific |
distribution to simulate the TCA+TWAS pipepine. |
=================================================
"

  "
  Generate SNPs and Gene Expression Data.
  @Params:
      N:          #samples
      D:          #SNP sequence length
      MAF:        #minor allele frequency
      M:          #gene expressions
      K:          #cell types
      p_slap:     sparsity. Proportion of gene not related to GE
      sigma_beta:         std of Beta (mu = X*beta+epsilon_{mu},beta gaussian)
      sigma_mu:           std of mu_z (mu = X*beta+epsilon_{mu},epsilon_mu gaussian)
      sigma_z:            std of Z (Z^{i}_{km}|mu^{i}_{km},epsilon^{i}_{km}~N{mu^{i}_{km},epsilon^{i}_{km})
      sigma_g:            std of Gene Expression
      alpha:              alpha for Cell Type Proportion, which follows dirichlet distribution
      Q                   number of traits
      train_proportion:   the proportion of training dataset
  @Shape information:
      This part tell us the shape information before we split them into training data and test data
      X:          N*D Matrix SNPs
      Z:          N*K*M Array cell type specific gene expression
      G:          M*N Matrix mixed gene expressions
      beta:       D*K*M Array effect size of SNPs on Z
      W:          N*K Matrix cell type proportion
      T:          Q Traits
      c1:         N*3
  @Returns:
      traing_data: ['X'=X, 'Z'=Z, 'G'=G, 'beta'=beta, 'W'=W, 'T'=T, 'c1'=c1]
      test_data:   ['X'=X, 'Z'=Z, 'G'=G, 'beta'=beta, 'W'=W, 'T'=T, 'c1'=c1]
  "


generate_data <- function(N=100,
                          D=2000,
                          MAF=0.1,
                          M=10,
                          K=6,
                          p_slab=0.5,
                          sigma_beta=0.3,
                          sigma_mu=0.3,
                          sigma_z=0.3,
                          sigma_g=0.01,
                          alpha=0.3,
                          Q=5,
                          train_proportion=0.7,
                          male_proportion=0.5,
                          smoking_proportion=0.2,
                          pc_num=2
                          ) {
    library('MCMCpack')
    
    #### names
    id_name = sapply(1:N, function(x) paste('SAMPLE',toString(x),sep = ''))
    cell_type=sapply(1:K, function(x) paste('Cell_type',x,sep = ''))
    snp_name=sapply(1:D, function(x) paste('SNP',toString(x),sep = ''))
    ge_name=sapply(1:M, function(x) paste('GE',toString(x),sep=''))
                   
    #### generate X
    X = sapply(1:D,function(x) rbinom(N, 2,0.1))
    while (qr(X)$rank<min(dim(X))){
        X = sapply(1:D,function(x) rbinom(N, 2,MAF))
    }
    
    colnames(X, do.NULL = TRUE, prefix = "col")
    colnames(X) <- snp_name
    rownames(X, do.NULL = TRUE, prefix = "col")
    rownames(X) <- id_name
    
    # mask a normal to form spikeslab
    beta = rnorm(D*K*M, mean=0, sd=sigma_beta)
    mask = rbinom(D*K*M, 1, 1-p_slab)
    beta = array(beta*mask, c(D,K,M))
    
    # \mu = X*\beta+\epsilon_mu
    epsilon_mu = array(rnorm(N*K*M, mean = 0, sd=sigma_mu), c(N,K,M))
    mu_z = lapply(c(1:M),function(x) X%*%beta[,,x])
    mu_z = array(as.numeric(unlist(mu_z)), dim=c(N,K,M)) + epsilon_mu
    # Simulate mu_jh that is shared across individuals
    mu <- array(runif(M*K, min=0, max=10), c(M,K))
    mu_stacked <- aperm(abind(lapply(1:N, function(x) mu), along = 3), c(3,2,1))
    mu <- mu_stacked
    # mu_z <- mu_stacked　# +mu_z
    
    #W = rdirichlet(N, rep(alpha,K))
    W <- rdirichlet(N, runif(K, 0, 1))
    colnames(W, do.NULL = TRUE, prefix = "col")
    colnames(W) <- cell_type
    rownames(W, do.NULL = TRUE, prefix = "col")
    rownames(W) <- id_name
  
    # Beta :QxM
    # T: NxQ
    # epsilon: NxQ
    
    ### generate C1 and C2
    male = matrix(rbinom(N, 1, 0.5),nrow=N)
    smoking = matrix(rbinom(N, 2, 0.2),nrow=N)
    smoking = (smoking - min(smoking))/(max(smoking))
    age =matrix(sapply(sapply(rnorm(N,50, 20), function(x) floor(x)), function(x)  if (x<0){x=20} else{x=x}),nrow=N) 
    age = (age-min(age))/(max(age)-min(age))
    c1= cbind(male, smoking, age)
    c2=matrix(rnorm(N*pc_num), nrow=N)
    rownames(c1, do.NULL = TRUE, prefix = "col")
    rownames(c1) <- id_name
    #colnames(c1, do.NULL = TRUE, prefix = "col")
    #colnames(c1) <- c('male','smoking','age', snp_name)
    #colnames(c1) <- snp_name
    rownames(c2, do.NULL = TRUE, prefix = "col")
    rownames(c2) <- id_name
    colnames(c2, do.NULL = TRUE, prefix = "col")
    colnames(c2) <- sapply(1:pc_num, function(x) paste('PC',toString(x),sep = ''))
                           
    ### Generating Z
    # sample z from mu_z and epsilon_za and C1
    # epsilon_z = array(rnorm(N*K*M,mean=0,sd=sigma_z), c(N,K,M))
    #sigma_z <- runif(K*M, 0, 1)
    #epsilon_z <- array(sapply(1:K*M, function(x) rnorm(N, mean=0, sd = sigma_z[x])),c(N,K,M))

    sigma_z = matrix(runif(M*K,0,0.3),M,K)
    epsilon_z = array(0,c(N,K,M))
    for(i in 1:M){
        for (j in 1:K){
            epsilon_z[,j,i] = rnorm(N, 0 ,sigma_z[i,j])
        }
    }
    Z = epsilon_z +mu_stacked
    dimnames(Z)[[1]] <- id_name
    dimnames(Z)[[2]] <- cell_type
    dimnames(Z)[[3]] <- ge_name
    # sample gama
    gama = array(sapply(1:K*M, function(x) runif(3, 0, 1)), dim = c(K, 3, M))
    c_gama = array(rep(0, N*K*M), c(N,K,M))
    for (i in 1:K) {
      c_gama[,i,] = c1 %*% gama[i,,]
    }
    Z = mu_z + Z# + c_gama
                        
    ### Generate Gene expressions
    G = sapply(c(1:M),function(x) rowSums(W*Z[,,c(x)]))
    epsilon_G = replicate(M,rnorm(N,mean=0,sd=sigma_g))
    G = G +epsilon_G
    T_t = runif(1)
    epsilon_G = replicate(Q,rnorm(N))*T_t*T_t
    beta_t = matrix(runif(Q*M),ncol=M)
    T = G%*%t(beta_t)+epsilon_G
    #generate delta
    delta = matrix(sapply(1:M*pc_num, function(x) runif(pc_num,0,1)), nrow = pc_num)
    c_delta = c2%*%delta
    G = G + c_delta
    
    rownames(G, do.NULL = TRUE, prefix = "col")
    rownames(G) <- id_name
    colnames(G, do.NULL = TRUE, prefix = "col")
    colnames(G) <- ge_name
    G = t(G)                      
    ### split into train and test data
    prop = floor(train_proportion*dim(X)[1])
    train_data = list('X'=X[c(1:prop),],
                      'Z'=Z[c(1:prop),,],
                      'G'=G[,c(1:prop)],
                      'beta'=beta,
                      'W'=W[c(1:prop),],
                      'T'=T[c(1:prop),],
                      'C1'=c1[c(1:prop),],
                      'C2'=c2[c(1:prop),],
                      'MU'=mu,
                      'SIGMA_Z'=sigma_z)
    test_data = list('X'=X[-c(1:prop),],
                     'Z'=Z[-c(1:prop),,],
                     'G'=G[,-c(1:prop)],
                     'beta'=beta,
                     'W'=W[-c(1:prop),],
                     'T'=T[-c(1:prop),],
                     'C1'=c1[-c(1:prop),],
                     'C2'=c2[-c(1:prop),],
                     'MU'=mu,
                     'SIGMA_Z'=sigma_z)
    
    return(list('train_data'=train_data,'test_data'=test_data))
}

[1] "\n=================================================\nThis file is used to generate data from specific |\ndistribution to simulate the TCA+TWAS pipepine. |\n=================================================\n"

[1] "\n  Generate SNPs and Gene Expression Data.\n  @Params:\n      N:          #samples\n      D:          #SNP sequence length\n      MAF:        #minor allele frequency\n      M:          #gene expressions\n      K:          #cell types\n      p_slap:     sparsity. Proportion of gene not related to GE\n      sigma_beta:         std of Beta (mu = X*beta+epsilon_{mu},beta gaussian)\n      sigma_mu:           std of mu_z (mu = X*beta+epsilon_{mu},epsilon_mu gaussian)\n      sigma_z:            std of Z (Z^{i}_{km}|mu^{i}_{km},epsilon^{i}_{km}~N{mu^{i}_{km},epsilon^{i}_{km})\n      sigma_g:            std of Gene Expression\n      alpha:              alpha for Cell Type Proportion, which follows dirichlet distribution\n      Q                   number of traits\n      train_proportion:   the proportion of training dataset\n  @Shape information:\n      This part tell us the shape information before we split them into training data and test data\n      X:          N*D Matrix SNPs\n      Z:          N*K*M Array cell type specific gene expression\n      G:          M*N Matrix mixed gene expressions\n      beta:       D*K*M Array effect size of SNPs on Z\n      W:          N*K Matrix cell type proportion\n      T:          Q Traits\n      c1:         N*3\n  @Returns:\n      traing_data: ['X'=X, 'Z'=Z, 'G'=G, 'beta'=beta, 'W'=W, 'T'=T, 'c1'=c1]\n      test_data:   ['X'=X, 'Z'=Z, 'G'=G, 'beta'=beta, 'W'=W, 'T'=T, 'c1'=c1]\n  "

# TCA Simulation
### I. Generate Data

In [211]:
D = 200
K = 4
M = 10
data=generate_data(N=1400,D=D,p_slab=0.5,M=M,K=K,MAF=0.18,train_proportion=0.7)
train_data=data[['train_data']]
test_data = data[['test_data']]
train_X=train_data[['X']]
train_W=train_data[['W']]
train_G = train_data[['G']]
train_c1=train_data[['C1']]
train_c2=train_data[['C2']]
train_Z=train_data[['Z']]
beta=train_data[['beta']]

test_X=test_data[['X']]
test_W=test_data[['W']]
test_G = test_data[['G']]
test_Z = test_data[['Z']]
test_c1=test_data[['C1']]
test_c2=test_data[['C2']]

In [212]:
print('whether D is a fat or a thin matrix')
N_ = dim(train_X)[1]
K_ = K
D_ = D
W_ = ncol(train_W)
if(N_ < (K_*D_+W_)){
    print('fat matrix, null space not null. semi definite. fail the test')
    paste(N_,'<',K_,'*',D_,'+',W_,'=',K_*D_+W_,sep=' ')
} else{
    print('tall matrx, null space might be zero. MAY/MAY NOT pass the test')
    paste(N_,'>',K_,'*',D_,'+',W_,'=',K_*D_+W_,sep=' ')
}

[1] "whether D is a fat or a thin matrix"
[1] "tall matrx, null space might be zero. MAY/MAY NOT pass the test"


[1] "979 > 4 * 200 + 4 = 804"

In [213]:
tca.mdl <- tca(X=train_G, W=train_W,C1=train_X,verbose=FALSE)

In [214]:
W = train_W# + rnm
k = ncol(W)
W_norms <- rowSums(W**2)**0.5
W_tilde <- W/t(repmat(W_norms,k,1))
C2 <- matrix(0, nrow=ncol(train_G), ncol=0)
train_X_ = train_X
n = nrow(W)
k = ncol(W)
p1 = ncol(train_X_)
C1_ = hadamard.prod(Reshape(Reshape(apply(W, 2, function(v) repmat(v,1,p1)), n*p1*k,1), n,p1*k), repmat(train_X_, 1, k))
C1_tilde <- C1_/t(repmat(W_norms,k*p1,1))
D = as.matrix(cbind(W_tilde,C2,C1_tilde))#5:ncol(C1_tilde)]))
Dmat <- t(D) %*% D
Dmat = round(Dmat,10)
print('W_tilde')
dim(W_tilde)
qr(W_tilde)$rank
print('C1_tilde')
dim(C1_tilde)
qr(C1_tilde)$rank
print('D')
dim(D)
qr(D)$rank
print('dependent columns of D')
#rankifremoved <- sapply(1:ncol(D), function (x) qr(D[,-x])$rank)
#rankifremoved
#which(rankifremoved == max(rankifremoved))
print('positive definite Dmat')
is.positive.definite(Dmat)

[1] "W_tilde"


[1] 979   4

[1] 4

[1] "C1_tilde"


[1] 979 800

...

[1] "D"


[1] 979 804

[1] 804

[1] "dependent columns of D"
[1] "positive definite Dmat"


[1] TRUE

In [215]:
rnm = matrix(rnorm(nrow(W)*ncol(W),mean=0,sd=0.1),nrow=nrow(W),ncol=ncol(W))

W = train_W# + rnm
k = ncol(W)
W_norms <- rowSums(W**2)**0.5
W_tilde <- W/t(repmat(W_norms,k,1))
C2 <- matrix(0, nrow=ncol(train_G), ncol=0)

# bad_rows = 10 34 40 80 117 121 217 226
bad_rows = array(0,ncol(train_X))
rankD = array(0,ncol(train_X))
dependent_row_number = list()
for (i in 1:ncol(train_X)){
    train_X_ = matrix(train_X[,i],nrow=nrow(train_X),ncol=1)
    n = nrow(W)
    k = ncol(W)
    p1 = ncol(train_X_)
    C1_ = hadamard.prod(Reshape(Reshape(apply(W, 2, function(v) repmat(v,1,p1)), n*p1*k,1), n,p1*k), repmat(train_X_, 1, k))
    C1_tilde <- C1_/t(repmat(W_norms,k*p1,1))
    D = as.matrix(cbind(W_tilde,C2,C1_tilde))#5:ncol(C1_tilde)]))
    Dmat <- t(D) %*% D
    Dmat = round(Dmat,10)
    if (!is.positive.definite(Dmat)){
        bad_rows[i] = 1
        rankD[i] = qr(D)$rank
        #rankifremoved <- sapply(1:ncol(D), function (x) qr(D[,-x])$rank)
        #dependent_row_number[length(dependent_row_number)+1] =  c(which(rankifremoved == max(rankifremoved)))
    }
}
print('all snps that fails postive definite tests')
# which(bad_rows==1)
rankD[rankD!=0] # Their D matrix's rank
# how many unique elements in 0/1/2/3 in these rows
unlist(apply(train_X[,c(which(bad_rows==1))],2,function(x) length(unique(x))))
#dependent_row_number
                                
                                
W = train_W + rnm
k = ncol(W)
W_norms <- rowSums(W**2)**0.5
W_tilde <- W/t(repmat(W_norms,k,1))
C2 <- matrix(0, nrow=ncol(train_G), ncol=0)

# bad_rows = 34, 10
bad_rows = array(0,ncol(train_X))
rankD = array(0,ncol(train_X))
dependent_row_number = list()
for (i in 1:ncol(train_X)){
    train_X_ = matrix(train_X[,i],nrow=nrow(train_X),ncol=1)
    n = nrow(W)
    k = ncol(W)
    p1 = ncol(train_X_)
    C1_ = hadamard.prod(Reshape(Reshape(apply(W, 2, function(v) repmat(v,1,p1)), n*p1*k,1), n,p1*k), repmat(train_X_, 1, k))
    C1_tilde <- C1_/t(repmat(W_norms,k*p1,1))
    D = as.matrix(cbind(W_tilde,C2,C1_tilde))#5:ncol(C1_tilde)]))
    Dmat <- t(D) %*% D
    Dmat = round(Dmat,10)
    if (!is.positive.definite(Dmat)){
        bad_rows[i] = 1
        rankD[i] = qr(D)$rank
        #rankifremoved <- sapply(1:ncol(D), function (x) qr(D[,-x])$rank)
        #dependent_row_number[length(dependent_row_number)+1] =  (which(rankifremoved == max(rankifremoved)))
    }

}
print('after random noise to weight, snps that fails')
which(bad_rows==1)
rankD[rankD!=0]

[1] "all snps that fails postive definite tests"


numeric(0)

integer(0)

[1] "after random noise to weight, snps that fails"


integer(0)

numeric(0)

### II. TCA model

In [199]:
D = 200
K = 4
M = 10
data=generate_data(N=5000,D=D,p_slab=0.5,M=M,K=K,MAF=0.18,train_proportion=0.7,sigma_g=0.3)
train_data=data[['train_data']]
test_data = data[['test_data']]
train_X=train_data[['X']]
train_W=train_data[['W']]
train_G = train_data[['G']]
train_c1=train_data[['C1']]
train_c2=train_data[['C2']]
train_Z=train_data[['Z']]
beta=train_data[['beta']]

test_X=test_data[['X']]
test_W=test_data[['W']]
test_G = test_data[['G']]
test_Z = test_data[['Z']]
test_c1=test_data[['C1']]
test_c2=test_data[['C2']]

In [200]:
rnm = matrix(rnorm(nrow(train_X)*ncol(train_X),mean=0,sd=0.1),nrow=nrow(train_X),ncol=ncol(train_X))
tca.mdl <- tca(X=train_G, W=train_W, C1=train_X, C2=train_c2,verbose=FALSE)
#tca.mdl <- tca(X=train_G, W=train_W,C1=train_c1,C2=train_c2,verbose=FALSE)
# tca.mdl <- tca(X=train_G, W=train_W,verbose=FALSE)
mu_hat=tca.mdl[['mus_hat']]
sigma_hat=tca.mdl[['sigmas_hat']]

# tca model summary statistics
z_hat=mu_hat#+sigma_hat

z_ = colSums(train_Z)/dim(train_Z)[[1]]
z_=t(z_)
cor.test(z_, z_hat)
cor(z_, z_hat)


	Pearson's product-moment correlation

data:  z_ and z_hat
t = 5.7698, df = 38, p-value = 1.178e-06
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.4724062 0.8202547
sample estimates:
      cor 
0.6833539 


,Cell_type1,Cell_type2,Cell_type3,Cell_type4
Cell_type1,0.9824847,-0.19472108,-0.28811985,-0.622327505
Cell_type2,-0.1487512,0.98390742,0.06534361,-0.212794786
Cell_type3,-0.4283274,0.09539096,0.92877144,0.001409613
Cell_type4,-0.3026210,-0.22275811,0.03637195,0.270610825


TCA models $$Z^{i}_{hj}\sim N(\mu_{hj},\sigma_{hj})$$ <br\> Same value across all samples. The test is the correlation between predicted $Z_{hj} \subseteq R_{MxK}$ and the real value

In [201]:
# correlation of the PCA prediction and the ground truth
# tca predicted z and the real z
for (gene_exp in c(1:dim(train_G)[[1]])){
    ge_name <- rownames(train_G)[gene_exp]
    tca.mdl.ge <- tcasub(tca.mdl,ge_name,log_file=NULL,verbose=FALSE)
    Z_hat <- tensor(t(train_G[gene_exp,]),tca.mdl.ge,log_file=NULL,verbose=FALSE)
    print(sapply(1:length(Z_hat), function(x) cor(unlist(Z_hat[x]),unlist(train_data[['Z']][,x,gene_exp]))))
}

[1] 0.8077872 0.7961064 0.8923311 0.2126965
[1] 0.6891876 0.6733700 0.8853929 0.1400134
[1] 0.55029543 0.49733547 0.74151487 0.01952763
[1] 0.5470466 0.5513850 0.7981101 0.1060033
[1] 0.609346194 0.540931856 0.816991929 0.008268835
[1] 0.7371731 0.6879743 0.8369080 0.0425594
[1] 0.6947268 0.6694169 0.8546228 0.1518756
[1] 0.562270931 0.550720472 0.887063255 0.003382284
[1] 0.78024756 0.79538859 0.85766763 0.07978051
[1] 0.7823855 0.7863335 0.8293514 0.1022495


In [202]:
colSums(tca.mdl$W)/dim(tca.mdl$W)[1]

Cell_type1 Cell_type2 Cell_type3 Cell_type4 
0.37815933 0.50887979 0.09997843 0.01298244

The above $MXK$ table is the correlation between tca predicted $z^{i}_{hj}$ and the real value.<br/>
Each $ELE_{hj}$ in the table is correlation between real $z_{hj} \subseteq R^{N}$ and tca predicted value  $z_{hj} \subseteq R^{N}$

In [203]:
cor.test(tca.mdl$sigmas_hat,train_data[['SIGMA_Z']])


	Pearson's product-moment correlation

data:  tca.mdl$sigmas_hat and train_data[["SIGMA_Z"]]
t = 0.74354, df = 38, p-value = 0.4617
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.1991889  0.4157513
sample estimates:
      cor 
0.1197509 


In [204]:
beta_train = aperm(train_data[['beta']],c(3,1,2))
gamma_p = array(tca.mdl$gammas_hat, dim = c(M, D, K))
cor_gamma_beta = matrix(0,M,K)
for (i in 1:M){
    for (j in 1:K){
        cor_gamma_beta[i,j] <- cor(beta_train[i,,j],gamma_p[i,,j])
    }
}
cor_gamma_beta

0.9827177,0.9890665,0.9222846,0.2481555800
0.9780070,0.9861204,0.9191402,0.1426478149
0.9836049,0.9916022,0.8720735,-0.0005804779
0.9689516,0.9895263,0.8970043,0.1001337762
0.9850551,0.9867198,0.8930724,-0.0024709921
0.9793592,0.9864795,0.9039294,0.0290540722
0.9839140,0.9896388,0.9082814,0.1459397122
0.9735277,0.9864784,0.9235121,-0.0064327625
0.9813457,0.9890656,0.8990125,0.0737133954
0.9837014,0.9918791,0.8870136,0.0967320961


In [205]:
c_gamma = array(0, c(dim(train_X)[1],K,M))
for (i in 1:K) {
   c_gamma[,i,] = train_X %*% t(gamma[,,i])
}
p1 = c_gamma+train_data[['MU']][1:dim(c_gamma),,]
cor.test(p1,train_Z)

p_mu = aperm(abind(lapply(1:dim(train_X)[1], function(x) tca.mdl$mus_hat), along = 3), c(3,2,1))
p1 = c_gamma+p_mu
cor.test(p1,train_Z)

Warning message in 1:dim(c_gamma):
“numerical expression has 3 elements: only the first used”


	Pearson's product-moment correlation

data:  p1 and train_Z
t = 363.93, df = 140000, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.6945396 0.6999231
sample estimates:
      cor 
0.6972412 



	Pearson's product-moment correlation

data:  p1 and train_Z
t = 235.37, df = 140000, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.5286983 0.5362046
sample estimates:
      cor 
0.5324619 


In [206]:
c_gamma_test = array(0, c(dim(test_X)[1],K,M))
for (i in 1:K) {
   c_gamma_test[,i,] = test_X %*% t(gamma[,,i])
}
p1 = c_gamma_test+test_data[['MU']][1:dim(c_gamma_test),,]
cor.test(p1,test_Z)

p_mu = aperm(abind(lapply(1:dim(test_X)[1], function(x) tca.mdl$mus_hat), along = 3), c(3,2,1))
p1 = c_gamma_test+p_mu
cor.test(p1,test_Z)

Warning message in 1:dim(c_gamma_test):
“numerical expression has 3 elements: only the first used”


	Pearson's product-moment correlation

data:  p1 and test_Z
t = 235.44, df = 59998, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.6888024 0.6971206
sample estimates:
      cor 
0.6929845 



	Pearson's product-moment correlation

data:  p1 and test_Z
t = 152.86, df = 59998, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.5236305 0.5351485
sample estimates:
      cor 
0.5294139 


### iii. Lasso & correlation

In [207]:
corrs.bulk <- numeric(nrow(train_G))
corrs <- matrix(0,nrow(train_G),ncol(tca.mdl$W))
corrs.real <- matrix(0,nrow(train_G),ncol(tca.mdl$W))
corrs.beta <- matrix(0,nrow(train_G),ncol(tca.mdl$W))
G.scaled <- scale(train_X,center=TRUE,scale=TRUE)

corrs.test.bulk <- numeric(nrow(test_G))
corrs.test <- matrix(0,nrow(test_G),ncol(tca.mdl$W))
corrs.test.real <- matrix(0,nrow(test_G),ncol(tca.mdl$W))
G.test.scaled <- scale(test_X,center=TRUE,scale=TRUE)

ge_names = rownames(train_G)
# loop through 10 gene expressions
for (i in 1:nrow(train_G)){

    # run bulk data regression first    
    glmnet.mdl.X.cv <- cv.glmnet(x=G.scaled,y=train_G[i,],nfolds=5)
    glmnet.mdl.X <- glmnet(x=G.scaled,y=train_G[i,],lambda=glmnet.mdl.X.cv$lambda.min)
    beta.full.X <- as.numeric(glmnet.mdl.X$beta)
    # extract non zero predictors and recorrelate
    predictors.X <- colnames(G.scaled)[which(beta.full.X!=0)]
    beta.X <- as.matrix(c(glmnet.mdl.X$a0,as.matrix(glmnet.mdl.X$beta[predictors.X,])))
    bias_one <- numeric(nrow(G.scaled))+1
    pred <- cbind(bias_one,G.scaled[,predictors.X]) %*% beta.X
    corrs.bulk[i] <- cor.test(train_G[i,],pred)[['estimate']][['cor']]
    pred.test <- cbind(numeric(nrow(G.test.scaled))+1,G.test.scaled[,predictors.X]) %*% beta.X
    corrs.test.bulk[i] <- cor.test(test_G[i,],pred.test)[['estimate']][['cor']]
    cat('gene expression ',i,' cell type:')

    ge_name <- ge_names[i]
    tca.mdl.ge <- tcasub(tca.mdl,ge_name,log_file=NULL,verbose=FALSE)
    Z_hat <- tensor(t(train_G[i,]),tca.mdl.ge,log_file=NULL,verbose=FALSE)
    #Z_hat.test <- tensor(t(test_G[i,]),tca.mdl.ge,log_file=NULL,verbose=FALSE)
    # cell type specific
    for (h in 1:ncol(tca.mdl$W)){
        cat('..',h)
        celltype <- colnames(tca.mdl$W)[h]
        glmnet.mdl.cv <- cv.glmnet(x=G.scaled,y=Z_hat[[h]],standardize=FALSE,alpha=1,nfolds=5)
        glmnet.mdl <- glmnet(x=G.scaled,y=Z_hat[[h]],standardize=FALSE,alpha=1,lambda=glmnet.mdl.cv$lambda.min)
        beta.full <- as.numeric(glmnet.mdl$beta)
        predictors <- colnames(G.scaled)[which(beta.full != 0)]
        beta <- as.matrix(c(glmnet.mdl$a0,as.matrix(glmnet.mdl$beta[predictors,])))
        Z_hat.h.pred <- cbind(numeric(nrow(G.scaled))+1,G.scaled[,predictors]) %*% beta
        corrs[i,h] <- cor(t(Z_hat[[h]]),Z_hat.h.pred)
        corrs.real[i,h] <- cor(train_Z[,h,i],Z_hat.h.pred)
        corrs.beta[i,h] <- cor(train_data[['beta']][,h,i],beta.full)
        
        Z_hat.h.test.pred <- cbind(numeric(nrow(G.test.scaled))+1,G.test.scaled[,predictors]) %*% beta
        #corrs.test[i,h] <- cor(Z_hat.test[,h,i],Z_hat.h.test.pred)
        corrs.test.real[i,h] <- cor(test_Z[,h,i],Z_hat.h.test.pred)
    }
    print('')
}

gene expression  1  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  2  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  3  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  4  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  5  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  6  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  7  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  8  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  9  cell type:.. 1.. 2.. 3.. 4[1] ""
gene expression  10  cell type:.. 1.. 2.. 3.. 4[1] ""


In [208]:
corrs.bulk
corrs.test.bulk

[1] 0.4059471 0.3659653 0.3835874 0.4861961 0.5429394 0.4668902 0.3946165
 [8] 0.4154513 0.5731874 0.5329600

[1] 0.3195288 0.2592887 0.2623129 0.3757179 0.4434095 0.3862063 0.2512027
 [8] 0.3691013 0.4718344 0.4385758

For each Gene Expression, fit a lasso model from SNPs to bulk level mixed GE. Final predictions are made
with the none-zero lasso predictors. The above vector is correlation between lasso's predicion and real value. <br/> The uppe rine being train statistics, lowerline test statistics

In [209]:
corrs.real
#corrs.test.real

0.9453641,0.9612620,0.9070996,0.211916736
0.9179348,0.9415840,0.9013875,0.140598356
0.9193619,0.9008419,0.8403678,0.019771155
0.9026811,0.9157163,0.8767548,0.107094699
0.9245241,0.9118108,0.8712207,0.007210433
0.9368232,0.9428232,0.8671899,0.044628295
0.9318626,0.9463059,0.8840634,0.151823202
0.8982425,0.9151180,0.9099859,0.004055627
0.9422043,0.9627611,0.8768885,0.078692776
0.9397827,0.9632895,0.8577969,0.100069338


The above $MXK$ table is the correlation between lasso predicted $z^{i}_{hj}$ and the real value.<br/>
Prediction is made with none-zero effect size estimators. Each $ELE_{hj}$ in the table is correlation
between real $z_{hj} \subseteq R^{N}$ and lasso predicted value $z_{hj} \subseteq R^{N}$. NA occurs when
lasso prediction has almost-zero variance
<br/> The uppe rine being train statistics, lowerline test statistics

In [210]:
corrs.beta

0.9746481,0.9806891,0.9211536,0.247710540
0.9559923,0.9700569,0.9197128,0.143900746
0.9340943,0.9137575,0.8655679,-0.001816845
0.9290723,0.9341807,0.8940879,0.100036535
0.9524584,0.9375970,0.8913610,-0.004330933
0.9655738,0.9677115,0.9036112,0.031672880
0.9598441,0.9648873,0.9056257,0.144359043
0.9352416,0.9444519,0.9225054,-0.006015329
0.9709010,0.9795497,0.8962832,0.072365966
0.9717284,0.9822356,0.8835417,0.092661759


The above $MXK$ table is the correlation between lasso effect-size $\beta$ and the real $\beta$.<br/>
Each $ELE_{hj}$ in the table is correlation between real $\beta_{hj}$ and lasso predicted value. NA occurs when
lasso prediction has almost-zero variance